In [30]:
import sys
import random

In [ ]:
from enum import Enum

class HeapType(Enum):
    MIN = "min"
    MAX = "max"

class HeapifyStrategy(Enum):
    UP = "up"
    DOWN = "down"

class CustomHeap:
    def __init__(self, heap_type=HeapType.MIN):
        self.heap = [None]  # 1-based indexing
        self.heap_type = heap_type  # Enum: MIN or MAX
        self.sorted_output = []  # Stores deleted elements in sorted order
        self.filename = ""

    def compare(self, a, b, compares):
        """Compare function based on heap type."""
        compares += 1
        return (a < b if self.heap_type == HeapType.MIN else a > b), compares

    def swap(self, i, j, swaps):
        """Swaps two elements and tracks swap count."""
        self.heap[i], self.heap[j] = self.heap[j], self.heap[i]
        swaps += 1
        return swaps
    
    def heapify_up(self, index):
        """Heapify up method."""
        compares, swaps = 0, 0
        while index > 1:
            parent = index // 2
            should_swap, compares = self.compare(self.heap[index], self.heap[parent], compares)
            if should_swap:
                swaps = self.swap(index, parent, swaps)
                index = parent
            else:
                break
        return {"compares": compares, "swaps": swaps}
    
    def heapify_down(self, index):
        """Heapify down method."""
        compares, swaps = 0, 0
        while 2 * index < len(self.heap):  # While left child exists
            left = 2 * index
            right = left + 1
            smallest_or_largest = left
            
            if right < len(self.heap):
                is_right_better, compares = self.compare(self.heap[right], self.heap[left], compares)
                if is_right_better:
                    smallest_or_largest = right
            
            should_swap, compares = self.compare(self.heap[smallest_or_largest], self.heap[index], compares)
            if should_swap:
                swaps = self.swap(index, smallest_or_largest, swaps)
                index = smallest_or_largest
            else:
                break                
        return {"compares": compares, "swaps": swaps}
    
    def build_heap(self, arr, strategy=HeapifyStrategy.DOWN):
        """Builds a heap from an array using either heapify-up or heapify-down."""
        self.heap = [None] + arr  # Maintain 1-based indexing
        total_compares, total_swaps = 0, 0
        self.print_heap(total_compares, total_swaps)
        
        if strategy == HeapifyStrategy.UP:
            for i in range(2, len(self.heap)):  
                result = self.heapify_up(i)
                total_compares += result["compares"]
                total_swaps += result["swaps"]
                self.print_heap(total_compares, total_swaps)
        else:  
            for i in range((len(self.heap) - 1) // 2, 0, -1):
                result = self.heapify_down(i)
                total_compares += result["compares"]
                total_swaps += result["swaps"]
                self.print_heap(total_compares, total_swaps)

        return {"compares": total_compares, "swaps": total_swaps}
    
    def get_heap(self):
        """Returns the heap content (excluding index 0)."""
        return self.heap[1:]

    def print_heap(self, compares, swaps):
        print(self.get_heap(), compares, swaps, self.sorted_output)
        print(self.graphviz(compares, swaps))

    def delete_root(self):
        """Deletes the root element and re-heapifies."""
        if len(self.heap) <= 1:
            return None, {"compares": 0, "swaps": 0}  # Heap is empty

        root_value = self.heap[1]
        self.heap[1] = self.heap[-1]  
        self.heap.pop()  
        self.sorted_output.append(root_value)  # Store deleted element
        
        if len(self.heap) > 1:
            result = self.heapify_down(1)
        else:
            result = {"compares": 0, "swaps": 0}

        return root_value, result
    
    def delete_heap(self):
        """Deletes all elements from the heap and returns them in sorted order."""
        self.sorted_output = []  # Reset output list
        total_compares, total_swaps = 0, 0
        self.print_heap(total_compares, total_swaps)
        
        while len(self.heap) > 1:
            _, result = self.delete_root()
            total_compares += result["compares"]
            total_swaps += result["swaps"]
            self.print_heap(total_compares, total_swaps)

        return self.sorted_output, {"compares": total_compares, "swaps": total_swaps}
    
    def graphviz(self, compares, swaps):
        heap = self.get_heap()
        n = len(heap)
    
        digraph_str = []
        digraph_str.append("digraph g {")
        digraph_str.append("node [shape=record];")       
    
        # Heap array representation with 0-based indexing
        value_repr = "|".join(f"{{{i+1}|{heap[i]}}}" for i in range(n)) 
        digraph_str.append(f'a [label="{value_repr}"];')

        # Output array (sorted elements)
        output_repr = "|".join(f"{val}" for val in self.sorted_output) if self.sorted_output else "None"
        digraph_str.append(f'c [label="Sorted Output: {output_repr}"];')

        # Label for comparisons and swaps
        digraph_str.append(f'b [label="Comparisons: {compares} | Swaps: {swaps}"];')
    
        # Nodes
        for i in range(n):
            digraph_str.append(f'{i} [label="{heap[i]}", xlabel="{i}"];')
    
        # Edges to represent heap structure
        for i in range(n):
            left = 2 * i + 1
            right = left + 1
            if left < n:
                digraph_str.append(f'edge [color=blue] {i} -> {left};')
            if right < n:
                digraph_str.append(f'edge [color=red] {i} -> {right};')

        digraph_str.append("}")
        final_digraph_str = "\n".join(digraph_str)
        return final_digraph_str





In [ ]:
import random
if __name__ == "__main__":
        n = 10
        a_rand = random.sample(range(-100, 101), 10)
        a_asc = [11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
        a_desc = sorted(a_asc, reverse=True)
        print(a_rand)
        print(a_asc)
        print(a_desc)

        print("-------Random Num: Build MIN Heap: Top to Bottom------")
        heap_min_a_asc = CustomHeap(HeapType.MIN)
        result = heap_min_a_asc.build_heap(a_asc, strategy=HeapifyStrategy.UP)

        print("-------Random Num: Build MIN Heap: Top to Bottom------")
        heap_min_a_desc = CustomHeap(HeapType.MIN)
        result = heap_min_a_desc.build_heap(a_desc, strategy=HeapifyStrategy.UP)

        print("-------Random Num: Build MIN Heap: Top to Bottom------")
        heap_min_a_rand = CustomHeap(HeapType.MIN)
        result = heap_min_a_rand.build_heap(a_rand, strategy=HeapifyStrategy.UP)


        print("-------Random Num: Build MAX Heap: Top to Bottom------")
        heap_max_a_asc = CustomHeap(HeapType.MIN)
        result = heap_max_a_asc.build_heap(a_asc, strategy=HeapifyStrategy.UP)

        print("-------Random Num: Build MAX Heap: Top to Bottom------")
        heap_min_a_desc = CustomHeap(HeapType.MIN)
        result = heap_min_a_desc.build_heap(a_desc, strategy=HeapifyStrategy.UP)

        print("-------Random Num: Build MAX Heap: Top to Bottom------")
        heap_min_a_rand = CustomHeap(HeapType.MIN)
        result = heap_min_a_rand.build_heap(a_rand, strategy=HeapifyStrategy.UP)

        
        

[-10, -81, -99, -26, 35, -86, 13, 19, -27, -89]
[11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
[20, 19, 18, 17, 16, 15, 14, 13, 12, 11]
-------Random Num: Build MIN Heap: Top to Bottom------
[11, 12, 13, 14, 15, 16, 17, 18, 19, 20] 0 0 []
digraph g {
node [shape=record];
a [label="{1|11}|{2|12}|{3|13}|{4|14}|{5|15}|{6|16}|{7|17}|{8|18}|{9|19}|{10|20}"];
c [label="Sorted Output: None"];
b [label="Comparisons: 0 | Swaps: 0"];
0 [label="11", xlabel="0"];
1 [label="12", xlabel="1"];
2 [label="13", xlabel="2"];
3 [label="14", xlabel="3"];
4 [label="15", xlabel="4"];
5 [label="16", xlabel="5"];
6 [label="17", xlabel="6"];
7 [label="18", xlabel="7"];
8 [label="19", xlabel="8"];
9 [label="20", xlabel="9"];
edge [color=blue] 0 -> 1;
edge [color=red] 0 -> 2;
edge [color=blue] 1 -> 3;
edge [color=red] 1 -> 4;
edge [color=blue] 2 -> 5;
edge [color=red] 2 -> 6;
edge [color=blue] 3 -> 7;
edge [color=red] 3 -> 8;
edge [color=blue] 4 -> 9;
}
[11, 12, 13, 14, 20, 16, 17, 18, 19, 15] 1 1 []
digraph g {
node [